In [ ]:
## Training models

%reset -f

import numpy as np
import scipy.io as spi
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow import keras
from keras.models import Model
from keras.layers.merge import concatenate
from tensorflow.keras import initializers, regularizers
from tensorflow.keras.layers import Dense, Input, Lambda, Dot
from tensorflow.keras.layers.experimental import preprocessing

dof = 76
data = spi.loadmat('data_400_samples_fs_100Hz_76_storey_NL_20FT.mat')
ytotal = data['y'][0:dof,:,:].T
f = data['f'][:,:].T

for disp_num in [5-1, 15-1, 35-1, 65-1, 75-1]: ## [10-1, 15-1, 35-1, 65-1, 75-1] for research paper equivalence
    print(disp_num)
    dt = 0.01
    min_t = 0
    max_t = 2

    t = np.arange(min_t, max_t, dt)
    lt = t.shape[0]

    y = ytotal[:,:,disp_num]

    print('Displacement number: '+str(disp_num+1)+', Training samples: 400, PPS: 100')
    train_n = 400

    pointspsamples = 100
    pointer = np.random.randint(0,lt-1,train_n*pointspsamples)
    
    UX = np.tile(f[0:train_n,0:-1:2],(pointspsamples,1))
    Y = np.zeros([train_n*pointspsamples,1])
    UXY = np.zeros([train_n*pointspsamples,1])

    for i in range(0,pointspsamples):
        for j in range(0,train_n):
            Y[int(train_n*i+j),0] = t[pointer[train_n*i+j]]
            UXY[int(train_n*i+j),0] = y[j,pointer[train_n*i+j]]
    
    ux_sensors = UX.shape[1]
    y_sensors = Y.shape[1]
    
        # # # # # # # # # # # # # # # # # # # #    
    
    with tf.device('/device:cpu:0'):       
        
        def layer_dense(nodes = 1, activation_func = 'relu',
                        kernel_init = initializers.GlorotNormal(),
                        name = None):
            layer = Dense(nodes, activation = activation_func,
                         kernel_initializer = kernel_init,
                         name = name)
            return layer

        def fn(x):
            y = tf.einsum("ij,ij->i", x[0], x[1])
            y = tf.expand_dims(y, axis=1)
            return y

        hidden_layer_nodes = 40

        layer_input_a = Input(shape = (ux_sensors, ), name = "input_a")    
        normalizer_a = preprocessing.Normalization(input_shape = [ux_sensors, ], name = '15_a')
        normalizer_a.adapt(UX)
        layer_15_a = normalizer_a(layer_input_a)
        layer_20_a = layer_dense(hidden_layer_nodes, name = '20_a')(layer_15_a)
        layer_25_a = layer_dense(hidden_layer_nodes, name = '25_a')(layer_20_a)

        layer_input_b = Input(shape = (y_sensors, ), name = "input_b")
        normalizer_b = preprocessing.Normalization(input_shape = [y_sensors, ], name = '15_b')
        normalizer_b.adapt(Y)
        layer_15_b = normalizer_b(layer_input_b)    
        
        layer_20_b = layer_dense(hidden_layer_nodes, name = '20_b')(layer_15_b)
        layer_25_b = layer_dense(hidden_layer_nodes, name = '25_b')(layer_20_b)

        layer_35 = Lambda(fn, output_shape = [None,1], name = '35')([layer_25_a, layer_25_b])
        layer_40 = Dense(1, activation = None,
                         kernel_initializer = initializers.GlorotNormal(),
                         name = '40')(layer_35)

        model = Model(inputs = [layer_input_a, layer_input_b], outputs = layer_40)
        model.summary()

        model.compile(loss = 'mse',
                      optimizer = tf.keras.optimizers.Adam(learning_rate=0.001),
                      metrics = 'mae')

        for i in range(0,5):
            print('iteration = '+str(i))
            model.fit({"input_a":UX, "input_b":Y}, UXY,
                      epochs = 1, verbose = 1, batch_size = 128)
            for j in range(0,999):
                model.fit({"input_a":UX, "input_b":Y}, UXY, epochs = 1, verbose = 0, batch_size = 128)

        string = 'model_save_76DOF_NONLINEAR_'+str(train_n)+'_DISP_'+str(disp_num+1)
        model.save(string)